# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import httpx
from selectolax.parser import HTMLParser
import re

# Get Data From vlr.gg

In [9]:
base_url = 'https://www.vlr.gg'

event_url = '/event/matches/1188/champions-tour-2023-lock-in-s-o-paulo/?series_id=all'
event_url = '/event/matches/1530/project-v-pro-weekly-6/?series_id=all'

res = httpx.get(base_url+event_url)

html = HTMLParser(res.text)

matches = html.css('a.wf-module-item')

completed_matches = [match for match in matches if match.css_first('div.ml-status').text() == 'Completed']


perfect_completed_matches = completed_matches[:21] + completed_matches[22:30] + completed_matches[31:]

for index,match in enumerate(completed_matches):
    match_res = httpx.get(match_url := f'{base_url}{match.attributes["href"]}')

    match_html = HTMLParser(match_res.text)

    match_name = match_html.css_first('title').text().strip().split(' | ')[0].split(' vs. ')

    messy_match_result = match_html.css_first('div.js-spoiler').text()
    match_result = ' ' + ''.join(re.findall("\S+",messy_match_result)) + ' '

    print(index, '>> ',end='')
    print(match_sum:=match_result.join(match_name))

    tables = match_html.css('table.wf-table-inset')

    for table in tables:
        for t in pd.read_html(table.html):
            display(transform_subcolumns_df(t).head(2))
            # transform_subcolumns_df(t)
    #         sample_df = t
    #         for key,df in get_clean_df(t).items():
    #             pass

0 >> Cologne Bubbele 0:1 Capybara Huggers


Type   Name  Team     R              ACS             K      ... HS%  FK      \
Side   Name  Team   all   atk   def  all  atk  def all atk  ... def all atk   
0      Lele  Colo  1.26  1.51  0.87  351  443  216  23  17  ...  39   6   3   
1     Vince  Colo  0.99  0.80  1.27  161  130  209  12   5  ...  22   0   0   

Type      FD         +/–.1          
Side def all atk def   all atk def  
0      3   3   2   1    +3  +1  +2  
1      0   1   1   0    -1  -1   0  

[2 rows x 38 columns]

Type   Name  Team     R              ACS             K      ... HS%  FK      \
Side   Name  Team   all   atk   def  all  atk  def all atk  ... def all atk   
0     stepa  Capy  1.81  2.21  1.55  318  435  241  25  15  ...  33   3   1   
1       Nex  Capy  1.40  1.20  1.54  262  217  292  20   6  ...  16   7   2   

Type      FD         +/–.1          
Side def all atk def   all atk def  
0      2   0   0   0    +3  +1  +2  
1      5   2   2   0    +5   0  +5  

[2 rows x 38 columns]

Type   Name  Team     R              ACS             K      ... HS%  FK      \
Side   Name  Team   all   atk   def  all  atk  def all atk  ... def all atk   
0      Lele  Colo  1.26  1.51  0.87  351  443  216  23  17  ...  39   6   3   
1     Vince  Colo  0.99  0.80  1.27  161  130  209  12   5  ...  22   0   0   

Type      FD         +/–.1          
Side def all atk def   all atk def  
0      3   3   2   1    +3  +1  +2  
1      0   1   1   0    -1  -1   0  

[2 rows x 38 columns]

Type   Name  Team     R              ACS             K      ... HS%  FK      \
Side   Name  Team   all   atk   def  all  atk  def all atk  ... def all atk   
0     stepa  Capy  1.81  2.21  1.55  318  435  241  25  15  ...  33   3   1   
1       Nex  Capy  1.40  1.20  1.54  262  217  292  20   6  ...  16   7   2   

Type      FD         +/–.1          
Side def all atk def   all atk def  
0      2   0   0   0    +3  +1  +2  
1      5   2   2   0    +5   0  +5  

[2 rows x 38 columns]

...

13 >> eSports Cologne 0:1 Sissi State Punks


Type  Name Team     R              ACS             K      ... HS%  FK          \
Side  Name Team   all   atk   def  all  atk  def all atk  ... def all atk def   
0      ara  ECO  1.14  1.03  1.28  263  242  289  19  10  ...  58   4   2   2   
1     zery  ECO  1.11  1.38  0.78  214  252  169  16  11  ...  14   0   0   0   

Type  FD         +/–.1          
Side all atk def   all atk def  
0      3   2   1    +1   0  +1  
1      0   0   0     0   0   0  

[2 rows x 38 columns]

Type    Name  Team     R              ACS             K      ... HS%  FK      \
Side    Name  Team   all   atk   def  all  atk  def all atk  ... def all atk   
0      KovaQ  Siss  1.40  1.19  1.58  295  256  328  24  10  ...  35   5   2   
1     GunnyX  Siss  1.29  1.57  1.06  310  363  267  23  13  ...  15   5   3   

Type      FD         +/–.1          
Side def all atk def   all atk def  
0      3   4   2   2    +1   0  +1  
1      2   1   1   0    +4  +2  +2  

[2 rows x 38 columns]

Type  Name Team     R              ACS             K      ... HS%  FK          \
Side  Name Team   all   atk   def  all  atk  def all atk  ... def all atk def   
0      ara  ECO  1.14  1.03  1.28  263  242  289  19  10  ...  58   4   2   2   
1     zery  ECO  1.11  1.38  0.78  214  252  169  16  11  ...  14   0   0   0   

Type  FD         +/–.1          
Side all atk def   all atk def  
0      3   2   1    +1   0  +1  
1      0   0   0     0   0   0  

[2 rows x 38 columns]

Type    Name  Team     R              ACS             K      ... HS%  FK      \
Side    Name  Team   all   atk   def  all  atk  def all atk  ... def all atk   
0      KovaQ  Siss  1.40  1.19  1.58  295  256  328  24  10  ...  35   5   2   
1     GunnyX  Siss  1.29  1.57  1.06  310  363  267  23  13  ...  15   5   3   

Type      FD         +/–.1          
Side def all atk def   all atk def  
0      3   4   2   2    +1   0  +1  
1      2   1   1   0    +4  +2  +2  

[2 rows x 38 columns]

# Cleaning Data

In [ ]:
def get_default(serie,func,dtype,default=np.nan):
    
    try:
        return serie.map(func).map(dtype)
    except AttributeError as e:
        return pd.Series([default] * serie.shape[0])

In [ ]:
series = pd.Series([1,2,3,4,5])
get_default(series,lambda x : x*2,int)

In [ ]:
def temp_clean_df(df,func):
    result_df = pd.DataFrame()
    
    result_df['Name'] = df['Unnamed: 0'].map(lambda n : n.split(' ')[0])
    result_df['Team'] = df['Unnamed: 0'].map(lambda n : n.split(' ')[1])
    result_df['R'] = get_default(df['R'],func,float)
    result_df['ACS'] = get_default(df['ACS'],func,int)
    result_df['K'] = get_default(df['K'],func,int)
    result_df['D'] = df['D'].map(lambda d : d[1:-1].strip()).map(func).map(int)
    result_df['A'] = df['A'].map(func).map(int)
    result_df['KD_Diff'] = df['+/–'].map(func).map(lambda d : int(d))
    result_df['KAST%'] = df['KAST'].map(func).map(lambda p : int(p[:-1])/100)
    result_df['ADR'] = df['ADR'].map(func).map(int)
    result_df['HS%'] = df['HS%'].map(func).map(lambda p : int(p[:-1])/100)
    result_df['FK'] = df['FK'].map(func).map(int)
    result_df['FD'] = df['FD'].map(func).map(int)
    result_df['FKFD_Diff'] = df['+/–.1'].map(func).map(int)
    
    
    
    return result_df

In [ ]:
def clean_df(df,func):
    result_df = pd.DataFrame()
    
    result_df['Name'] = df['Unnamed: 0'].map(lambda n : n.split(' ')[0])
    result_df['Team'] = df['Unnamed: 0'].map(lambda n : n.split(' ')[1])
    result_df['R'] = df['R'].map(func).map(float)
    result_df['ACS'] = df['ACS'].map(func).map(int)
    result_df['K'] = df['K'].map(func).map(int)
    result_df['D'] = df['D'].map(lambda d : d[1:-1].strip()).map(func).map(int)
    result_df['A'] = df['A'].map(func).map(int)
    result_df['KD_Diff'] = df['+/–'].map(func).map(lambda d : int(d))
    result_df['KAST%'] = df['KAST'].map(func).map(lambda p : int(p[:-1])/100)
    result_df['ADR'] = df['ADR'].map(func).map(int)
    result_df['HS%'] = df['HS%'].map(func).map(lambda p : int(p[:-1])/100)
    result_df['FK'] = df['FK'].map(func).map(int)
    result_df['FD'] = df['FD'].map(func).map(int)
    result_df['FKFD_Diff'] = df['+/–.1'].map(func).map(int)
    
    
    
    return result_df

In [ ]:
def get_clean_df(df):
    result = {}
    def get_all(s):
        try:
            return s.split(' ')[0]
        except AttributeError:
            return '00'
    
    def get_atk(s):
        try:
            return s.split(' ')[2]
        except AttributeError:
            return '00'
    
    def get_def(s):
        try:
            return s.split(' ')[4]
        except AttributeError:
            return '00'

    for side in ['all','atk','def']:
        result[side] = clean_df(df,eval(f'get_{side}'))
        
    return result

# Testing

In [ ]:
sample_match = completed_matches[21]

In [ ]:
match_res = httpx.get(match_url := f'{base_url}{sample_match.attributes["href"]}')

match_html = HTMLParser(match_res.text)

match_name = match_html.css_first('title').text().strip().split(' | ')[0].split(' vs. ')

messy_match_result = match_html.css_first('div.js-spoiler').text()
match_result = ' ' + ''.join(re.findall("\S+",messy_match_result)) + ' '
print(match_sum:=match_result.join(match_name))

tables = match_html.css('table.wf-table-inset')

for table in tables:
    for t in pd.read_html(table.html):
        # display(transform_subcolumns_df(t))
        og_sample_df = t
        # for key,df in get_clean_df(t).items():
        #     pass

In [3]:
def extract_data(data) -> str:
    try:
        list_data = re.findall('[\d+-.]+',data)
        if (ll:=len(list_data)) == 0:
            return 'Nan Nan Nan'
        elif ll < 3:
            list_data += ['0'] * (3-ll)
        return ' '.join(list_data)
    except TypeError:
        return f'{data} Nan Nan'

In [4]:
sample_df.D.map(extract_data)

NameError: name 'sample_df' is not defined

In [5]:
nan_list = re.findall('[\d+-.]+','Nan Nan Nan')
len(nan_list)

0

In [2]:
def transform_subcolumns_df(df):
    filled_df =df.fillna('Nan Nan Nan').astype('object')
    
    name = filled_df.iloc[:,0].map(lambda s : s.split(' ')[0])
    team = filled_df.iloc[:,0].map(lambda s : s.split(' ')[1])
    
    filled_df.iloc[:,0] = name
    filled_df.iloc[:,1] = team
    
    formatted_df = filled_df.rename(columns={'Unnamed: 0':'Name','Unnamed: 1':'Team'})
    
    
    
    # formatted_df.iloc[:,2:] = formatted_df.iloc[:,2:].applymap(lambda x :  ' '.join(re.findall("[\d+-.]+",x,re.A)))
    formatted_df.iloc[:,2:] = formatted_df.iloc[:,2:].applymap(extract_data)
    
    
    
    new_columns = []
    for c in formatted_df.columns[2:]:
        for side in ['all','atk','def']:
            new_columns.append((c,side))

    new_columns = np.array(new_columns).reshape(-1,3,2)
    
    
    result_df = pd.DataFrame(formatted_df.iloc[:,:2])
    
    
    result_df.columns = [('Name','Name'),('Team','Team')]
    
    
    for nc,c in zip(new_columns,formatted_df.iloc[:,2:].columns):
        result_df[list(map(lambda x: tuple(x),list(nc)))] = formatted_df[c].str.split(' ',expand=True)
        
    
    result_df.columns = pd.MultiIndex.from_tuples(result_df.columns,name=['Type','Side'])
    
    return result_df

In [ ]:
sample_df = og_sample_df.copy()

In [ ]:
sample_df

In [ ]:
transform_subcolumns_df(sample_df)

In [ ]:
sample_df[['K','D','A']].info()

In [ ]:
sample_df.D.map(extract_data)

In [ ]:
sample_df = sample_df.fillna('0').astype('object')

In [ ]:
sample_df

In [ ]:
sample_df.info()

In [ ]:
sample_df['R'].map(lambda x : re.findall("[\d+-.]+",x))

In [ ]:
sample_df['R']

## Sub Column Approach

In [ ]:
sample_df

In [ ]:
name = sample_df.iloc[:,0].map(lambda s : s.split(' ')[0])
team = sample_df.iloc[:,0].map(lambda s : s.split(' ')[1])

In [ ]:
sample_df=sample_df.rename(columns={'Unnamed: 0':'Name','Unnamed: 1':'Team'})

In [ ]:
sample_df.iloc[:,0] = name
sample_df.iloc[:,1] = team
display(sample_df)

In [ ]:
def extract_data(data) -> str:
    list_data = re.findall("[\d+-.]+",data)
    
    if len(list_data) < 3:
        result = '0 0 0'
    else:
        result = ' '.join(list_data)
    
    return result

In [ ]:
sample_df['R'].map(extract_data)

In [ ]:
sample_df.iloc[:,2:] = sample_df.iloc[:,2:].applymap(lambda x :  ' '.join(re.findall("[\d+-.]+",x,re.A)))
sample_df

In [ ]:
new_columns = []
for c in sample_df.columns[2:]:
    for side in ['all','atk','def']:
        new_columns.append((c,side))
    
new_columns = np.array(new_columns).reshape(-1,3,2)

In [ ]:
for i in new_columns[::3]:
    print(i)

In [ ]:
sample_df

In [ ]:
new_sample_df = pd.DataFrame(sample_df.iloc[:,:2])

In [ ]:
new_sample_df.columns = [('Name','Name'),('Team','Team')]

In [ ]:
for nc,c in zip(new_columns,sample_df.iloc[:,2:].columns):
    new_sample_df[list(map(lambda x: tuple(x),list(nc)))] = sample_df[c].str.split(' ',expand=True)

In [ ]:
new_sample_df.columns = pd.MultiIndex.from_tuples(new_sample_df.columns,name=['Type','Side'])

In [ ]:
new_sample_df